[How to stream runnables](https://python.langchain.com/docs/how_to/streaming/)

In [ ]:
%pip install python-dotenv

In [14]:
from dotenv import load_dotenv

load_dotenv()

True

#### Using Stream

**LLMs and Chat Models**

In [1]:
%pip install -qU "langchain[openai]"


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [3]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors| such| as| time| of| day|,| weather| conditions|,| and| atmospheric| particles|.| Generally|,| during| a| clear| day|,| the| sky| appears| blue| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| This| scattering| causes| shorter| wavelengths| of| light| (|blue|)| to| be| scattered| more| than| longer| wavelengths| (|red|).| During| sunrise| and| sunset|,| the| sky| can| take| on| hues| of| orange|,| pink|,| and| red| due| to| the| angle| of| the| sun| and| the| increased| distance| the| light| travels| through| the| atmosphere|.| On| cloudy| days|,| the| sky| may| appear| gray| or| white|.||

In [4]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|The| color| of| the| sky| typically| appears| blue| during| the| day| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| This| scattering| causes| shorter| blue| wavelengths| of| light| to| be| more| prominent|.| However|,| the| sky| can| change| colors| depending| on| the| time| of| day|,| weather| conditions|,| and| other| factors|.| For| example|,| it| can| appear| gray| on| cloudy| days|,| orange| or| pink| during| sunrise| and| sunset|,| and| even| black| at| night| when| the| sun| is| down|.||

In [8]:
chunks[0]

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-0b269c60-e284-4e76-abe8-56b6ae4f042f')

In [6]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='The color of the', additional_kwargs={}, response_metadata={}, id='run-0b269c60-e284-4e76-abe8-56b6ae4f042f')

**Chains**

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)


|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|-uns|aturated|!||

**Working with Input Streams**

主要讲对于像json这样的数据格式，该如何进行stream处理。

langchain 里提到：the parser needs to operate on the input stream, and attempt to "auto-complete" the partial json into a valid state.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467

Any steps in the chain that operate on finalized inputs rather than on input streams can break streaming functionality via stream or astream.

也就是说中间的组件必须具备处理 input stream 的能力，否则就会打断 stream。

In [10]:
from langchain_core.output_parsers import JsonOutputParser

# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

所以后续给了一个修正的例子：

A generator function (a function that uses yield) allows writing code that operates on input streams

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# 大模型里提到入参 input_stream 是一个异步迭代器
async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""

    print(input_stream)

    country_names_so_far = set()

    async for input in input_stream:

        print(input)
        
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                # 注意 yield 的用法
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

<async_generator object tee_peer at 0x11d817be0>
{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
France|{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
Spain|{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France'

**Non-streaming components**

有些组件不支持 streaming：

In [ ]:
%pip install faiss-cpu

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='2ea5d1d7-e339-4fa7-837a-0418fde826b1', metadata={}, page_content='harrison worked at kensho'),
  Document(id='52b39def-80ae-4644-b5a5-6360074727b9', metadata={}, page_content='harrison likes spicy food')]]

但如果chain中既有non-streaming组件也有streaming组件，很多情况下也是支持stream输出的，在最后一个non-streaming组件之后再stream输出部分output：

In [21]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [22]:
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

|H|arrison| worked| at| Kens|ho|.| Kens|ho| is| a| cutting|-edge| tech| company| known| for| its| innovative| approach| to| data| analytics|.| The| team| at| Kens|ho| enjoys| a| lively| atmosphere| where| creativity| thrives| and| collaboration| is| encouraged|.| Employees| often| gather| for| brainstorming| sessions| that| frequently| result| in| groundbreaking| ideas| and| solutions|.||

#### Using Stream Events

使用 astream_events 有几点建议：

* Use async throughout the code to the extent possible (e.g., async tools etc)

* Propagate callbacks if defining custom functions / runnables

* Whenever using runnables without LCEL, make sure to call .astream() on LLMs rather than .ainvoke to force the LLM to stream tokens.

供参考的 events，见：https://python.langchain.com/docs/how_to/streaming/#event-reference

**Chat Model**

In [25]:
events = []
async for event in model.astream_events("hello"):
    events.append(event)

In [26]:
events

[{'event': 'on_chat_model_start',
  'data': {'input': 'hello'},
  'name': 'ChatOpenAI',
  'tags': [],
  'run_id': '0cc274c4-0ed8-488d-b86a-e2c58b358eec',
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': None},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '0cc274c4-0ed8-488d-b86a-e2c58b358eec',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': None},
  'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-0cc274c4-0ed8-488d-b86a-e2c58b358eec')},
  'parent_ids': []},
 {'event': 'on_chat_model_stream',
  'run_id': '0cc274c4-0ed8-488d-b86a-e2c58b358eec',
  'name': 'ChatOpenAI',
  'tags': [],
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'ls_temperature': None},
  'data': {'chunk': A

**Chain**

In [27]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

events = [
    event
    async for event in chain.astream_events(
        "output a list of the countries france, spain and japan and their populations in JSON format. "
        'Use a dict with an outer key of "countries" which contains a list of countries. '
        "Each country should have the key `name` and `population`",
    )
]

In [28]:
events[:3]

[{'event': 'on_chain_start',
  'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'},
  'name': 'RunnableSequence',
  'tags': [],
  'run_id': '801abc9f-55d2-4c6a-bd8b-75e1a7327a3e',
  'metadata': {},
  'parent_ids': []},
 {'event': 'on_chat_model_start',
  'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}},
  'name': 'ChatOpenAI',
  'tags': ['seq:step:1'],
  'run_id': '31591450-70ac-47e8-bf32-e08cb963eca1',
  'metadata': {'ls_provider': 'openai',
   'ls_model_name': 'gpt-4o-mini',
   'ls_model_type': 'chat',
   'l

In [31]:
events[-3:]

[{'event': 'on_chat_model_end',
  'data': {'output': AIMessageChunk(content='Here is the requested information in JSON format:\n\n```json\n{\n  "countries": [\n    {\n      "name": "France",\n      "population": 65273511\n    },\n    {\n      "name": "Spain",\n      "population": 46754778\n    },\n    {\n      "name": "Japan",\n      "population": 126476461\n    }\n  ]\n}\n```\n\nNote: The population figures are estimates as of 2023 and may vary.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306'}, id='run-31591450-70ac-47e8-bf32-e08cb963eca1'),
   'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}},
  'run_id': '31591450-7

提取出 model 和 parser 的events，忽略 start、end 以及 chain 相关的events：

In [30]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' requested'
Chat model chunk: ' data'
Chat model chunk: ' in'
Chat model chunk: ' JSON'
Chat model chunk: ' format'
Chat model chunk: ':\n\n'
Chat model chunk: '```'
Chat model chunk: 'json'
Chat model chunk: '\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
Chat model chunk: 'countries'
Chat model chunk: '":'
Chat model chunk: ' [\n'
Parser chunk: {'countries': []}
Chat model chunk: '   '
Chat model chunk: ' {\n'
Parser chunk: {'countries': [{}]}
Chat model chunk: '     '
...


**Filtering Events**

可以过滤 events，或者根据 name，或者根据 tags，或者根据 type。

根据 name：

In [32]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    include_names=["my_parser"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': 'a8619289-ff01-46d7-9cc6-5f9b30cfcfc6', 'metadata': {}, 'parent_ids': ['46b9bae7-7f8e-4e5a-9100-c4d23118c162']}
{'event': 'on_parser_stream', 'run_id': 'a8619289-ff01-46d7-9cc6-5f9b30cfcfc6', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['46b9bae7-7f8e-4e5a-9100-c4d23118c162']}
{'event': 'on_parser_stream', 'run_id': 'a8619289-ff01-46d7-9cc6-5f9b30cfcfc6', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['46b9bae7-7f8e-4e5a-9100-c4d23118c162']}
{'event': 'on_parser_stream', 'run_id': 'a8619289-ff01-46d7-9cc6-5f9b30cfcfc6', 'name': 'my_parse

根据 type：

In [33]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    include_types=["chat_model"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chat_model_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'model', 'tags': ['seq:step:1'], 'run_id': '9a4c5daa-bcde-40cf-92a6-e8abbfb3d256', 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': None}, 'parent_ids': ['e2c14543-d794-4194-96ac-bb6f5d4ebda7']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-9a4c5daa-bcde-40cf-92a6-e8abbfb3d256')}, 'run_id': '9a4c5daa-bcde-40cf-92a6-e8abbfb3d256', 'name': 'model', 'tags': ['seq:step:1'], 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': None}, 'parent_ids': ['e2c14543-d794-4194-96ac-bb6f5d4ebda7']}
{'event':

**By Tags**

有风险提示：

Tags are inherited by child components of a given runnable.

If you're using tags to filter, make sure that this is what you want.

In [34]:
chain = (model | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '9983290a-d740-4c06-a7e9-07a30acfa68f', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatOpenAI', 'tags': ['seq:step:1', 'my_chain'], 'run_id': 'f458ebee-71cf-4fc3-b37e-eb97a281e964', 'metadata': {'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperatur

**Non-streaming components**

虽然在使用astream时，此类组件可能会中断最终输出的流，但astream_events仍将从支持流的中间步骤生成流事件！

In [35]:
# Function that does not support streaming.
# It operates on the finalizes inputs rather than
# operating on the input stream.
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = (
    model | JsonOutputParser() | _extract_country_names
)  # This parser only works with OpenAI right now

下面这个例子验证 astream 不能很好的工作：

In [36]:
async for chunk in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(chunk, flush=True)

['France', 'Spain', 'Japan']


然后验证 astream_events 仍然在 model 和 parser 阶段可见：

In [37]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: '```'
Chat model chunk: 'json'
Chat model chunk: '\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
Chat model chunk: 'countries'
Chat model chunk: '":'
Chat model chunk: ' [\n'
Parser chunk: {'countries': []}
Chat model chunk: '   '
Chat model chunk: ' {\n'
Parser chunk: {'countries': [{}]}
Chat model chunk: '     '
Chat model chunk: ' "'
Chat model chunk: 'name'
Chat model chunk: '":'
Chat model chunk: ' "'
Parser chunk: {'countries': [{'name': ''}]}
Chat model chunk: 'France'
Parser chunk: {'countries': [{'name': 'France'}]}
Chat model chunk: '",\n'
Chat model chunk: '     '
Chat model chunk: ' "'
Chat model chunk: 'population'
...


**Propagating Callbacks**

【caution】

If you're using invoking runnables inside your tools, you need to propagate callbacks to the runnable; otherwise, no stream events will be generated.

【note】

When using RunnableLambdas or @chain decorator, callbacks are propagated automatically behind the scenes.

下面举了两个例子：第一个是 bad case，第二个是 good case。但是我真没看出有啥区别！

In [39]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]


reverse_word = RunnableLambda(reverse_word)


@tool
def bad_tool(word: str):
    """Custom tool that doesn't propagate callbacks."""
    return reverse_word.invoke(word)


async for event in bad_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': '520f6fb5-169f-4e7f-b22a-4c1444263d3a', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': '41517c00-d353-474b-876d-3eb0f7fa36e0', 'metadata': {}, 'parent_ids': ['520f6fb5-169f-4e7f-b22a-4c1444263d3a']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': '41517c00-d353-474b-876d-3eb0f7fa36e0', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['520f6fb5-169f-4e7f-b22a-4c1444263d3a']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': '520f6fb5-169f-4e7f-b22a-4c1444263d3a', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [40]:
@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': '9e0cef51-a5b4-4c14-99fb-38c89659d393', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'fcfe5b2c-f234-4c00-90ae-8d812f6ca637', 'metadata': {}, 'parent_ids': ['9e0cef51-a5b4-4c14-99fb-38c89659d393']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': 'fcfe5b2c-f234-4c00-90ae-8d812f6ca637', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['9e0cef51-a5b4-4c14-99fb-38c89659d393']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': '9e0cef51-a5b4-4c14-99fb-38c89659d393', 'name': 'correct_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


然后下面两个例子，列举了使用 Lambdas or @chains 的情况：

In [41]:
from langchain_core.runnables import RunnableLambda


async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '45e5abb1-31d2-45a4-8ba2-564a9b4f0c89', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': '83d1d739-6bfb-4239-8659-4a6c53ba765c', 'metadata': {}, 'parent_ids': ['45e5abb1-31d2-45a4-8ba2-564a9b4f0c89']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': '83d1d739-6bfb-4239-8659-4a6c53ba765c', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['45e5abb1-31d2-45a4-8ba2-564a9b4f0c89']}
{'event': 'on_chain_stream', 'run_id': '45e5abb1-31d2-45a4-8ba2-564a9b4f0c89', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '45e5abb1-31d2-45a4-8ba2-564a9b4f0c89', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [42]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '26bff890-8682-4da7-93c3-6a0f84116c5d', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'f459d024-d526-4ce0-9cb3-91f9ab3d19f7', 'metadata': {}, 'parent_ids': ['26bff890-8682-4da7-93c3-6a0f84116c5d']}
{'event': 'on_chain_end', 'data': {'output': '4321', 'input': '1234'}, 'run_id': 'f459d024-d526-4ce0-9cb3-91f9ab3d19f7', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['26bff890-8682-4da7-93c3-6a0f84116c5d']}
{'event': 'on_chain_stream', 'run_id': '26bff890-8682-4da7-93c3-6a0f84116c5d', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '26bff890-8682-4da7-93c3-6a0f84116c5d', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}
